In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from glob import glob
from typing import Dict, List
import os
import glob
import subprocess
import pytorch_hpc

In [ ]:
me = subprocess.run(["whoami"], capture_output=True).stdout.decode('utf').strip()

In [ ]:
def extract_scalar_data(fname: str, report_info=True) -> Dict[str,List]:
    """Returns a dictionary of str:List pairs that correspond to 
    scalar tags and their list of ScalarEvents
    
    Parameters
    ----------
    fname:
        Full path to the desired tensorboard events.out file
        
    Returns
    -------
    data_dict:
        Dictionary of tag:List[Scalar.Event]
    """
    events = EventAccumulator(fname)
    events.Reload()
    scalar_tags = events.Tags()['scalars']
    data_dict = {}
    for tag in scalar_tags:
        data_dict[tag] = events.Scalars(tag)

    if report_info:
        wts = [i.wall_time for i in data_dict["epoch"]] 
        avg = np.average(np.diff(wts))
        std = np.std(np.diff(wts))
        total = wts[-1] - wts[1]
        print(f"Summary for {fname}:")
        print("====================")
        print(f"Total training time: {total:.4f} (s)")
        print(f"Average training time: {avg:.4f} +/- {std:.2f} (s)")
        print("\n")
        
    return data_dict

def get_last_logfile(directory: str) -> str:
    """Grabs the last tensorboard eventfile in the specified directory"""

    files = list(glob.glob(directory + "/events*"))
    if len(files) == 0:
        print(f"No event files found in {directory}")
        return None
        
    files = [f for f in files if os.path.isfile(os.path.join(directory, f))]

    files_sorted = sorted(files, key=lambda f: os.path.getmtime(os.path.join(directory, f)))
    return files_sorted[-1]

In [ ]:
work = f"/home/woody/k_t70h/{me}"

cpu_logfile = get_last_logfile(f"{work}/NHR-AI-2025/pytorch_tests/fashion_mnist_cpu/tensorboard")
gpu_logfile = get_last_logfile(f"{work}/NHR-AI-2025/pytorch_tests/fashion_mnist_gpu/tensorboard")
cpu_data = extract_scalar_data(cpu_logfile)
gpu_data = extract_scalar_data(gpu_logfile)

In [ ]:
datas = [cpu_data, gpu_data]
colors = ["blue","red"]
labels = ['CPU','GPU']

for data_dict, color, label in zip(datas, colors, labels):
    vals = [ev.value for ev in data_dict['validation_loss']]
    trains  = [ev.value for ev in data_dict['training_loss']]
    plt.plot(trains, label=f"{label} Train", color=color, linestyle='--')
    plt.plot(vals, label=f"{label} Val", color=color)

plt.ylabel(r"Loss")
plt.xlabel("Epoch")
plt.legend(loc='best')
plt.show()

In [ ]:
help(pytorch_hpc.nn.models.ConvolutionClassifier)